In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import re
import sys
from splinter import Browser
import os
import csv
import threading
import requests
import pymongo
import pandas as pd
import string

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.Hotels_db
collection = db.items

In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)


In [5]:
# URL of page to be scraped
url_list= [line.strip() for line in open("Book2.txt", 'r')]




for link in url_list:
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    browser.visit(link)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find_all('section', class_="hotel-wrap")
    post = {}
    source_loaded = 'Hotels.com'
        
    for result in results:
    # Error handling
        try:
            # Identify and return title of listing
            address = result.find('address', class_="contact").text

            # # Identify and return price of listing
            price = result.find('a', class_="price-link").text


            # Print results only if postal code and price are available
            if (address,price):
                print('-------------')
                address_split = address.split(sep=',')
                if len(address_split) > 6:
                    postal_code = address_split[4]
                    print(f'Postal code: {postal_code}')
                
                else :
                    postal_code = address_split[3]
                    print(f'Postal code: {postal_code}')

                prices = price.split(sep='CAD')
                #Checking the lowest price 
                if len(prices) > 2:
                    new_prices = prices[1]
                    print(f'new price is: {new_prices}')

                else :
                    new_prices = prices[0]
                    print(f'new price is: {new_prices}')
                # Dictionary to be inserted as a MongoDB document
                post = {
                    'postal_code': postal_code,
                    'price': new_prices,
                    'source_loaded': source_loaded
                    }
                collection.insert_one(post)
                
        except AttributeError as e:
            print(e)
        

-------------
Postal code:  M5T2V4
new price is: $119 
-------------
Postal code:  M5T2S3
new price is: $139 
-------------
Postal code:  M5T1E8
new price is: $89 
-------------
Postal code:  M5T2C2
new price is: $144 
-------------
Postal code:  M5V2V4
new price is: $259 
-------------
Postal code:  M1H2Z1
new price is: $139 
-------------
Postal code:  M1H2Z1
new price is: $140 
-------------
Postal code:  M1T-3G2
new price is: $154 
-------------
Postal code:  Canada
new price is: $78 
-------------
Postal code:  M1M1P8
new price is: $105 
-------------
Postal code:  M9C5K5
new price is: $179 
-------------
Postal code:  M9C 5K5
new price is: $149 
-------------
Postal code:  M9C 5K5
new price is: $119 
-------------
Postal code:  L4W4X2
new price is: $139 
-------------
Postal code:  L4W5R2
new price is: $139 
-------------
Postal code:  M9W 5E7
new price is: $117 
-------------
Postal code:  M9W5E8
new price is: $130 
-------------
Postal code:  M9W 5E8
new price is: $199 
-------

WebDriverException: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"Cannot navigate to invalid URL"}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.34.522940 (1a76f96f66e3ca7b8e57d503b4dd3bccfba87af1),platform=Windows NT 10.0.17763 x86_64)


In [8]:
#remove all the listing from db
#db.items.remove()
#items in MongoDB collection
listings = db.items.find()
listings



In [7]:
# Display items in MongoDB collection
listings = db.items.find()

for listing in listings:
    print(listing)

{'_id': ObjectId('5cba35947fbad2066479859f'), 'postal_code': ' M5T2V4', 'price': '$119 ', 'source_loaded': 'Hotels.com'}
{'_id': ObjectId('5cba35947fbad206647985a0'), 'postal_code': ' M5T2S3', 'price': '$139 ', 'source_loaded': 'Hotels.com'}
{'_id': ObjectId('5cba35947fbad206647985a1'), 'postal_code': ' M5T1E8', 'price': '$89 ', 'source_loaded': 'Hotels.com'}
{'_id': ObjectId('5cba35947fbad206647985a2'), 'postal_code': ' M5T2C2', 'price': '$144 ', 'source_loaded': 'Hotels.com'}
{'_id': ObjectId('5cba35947fbad206647985a3'), 'postal_code': ' M5V2V4', 'price': '$259 ', 'source_loaded': 'Hotels.com'}
{'_id': ObjectId('5cba359c7fbad206647985a4'), 'postal_code': ' M1H2Z1', 'price': '$139 ', 'source_loaded': 'Hotels.com'}
{'_id': ObjectId('5cba359c7fbad206647985a5'), 'postal_code': ' M1H2Z1', 'price': '$140 ', 'source_loaded': 'Hotels.com'}
{'_id': ObjectId('5cba359c7fbad206647985a6'), 'postal_code': ' M1T-3G2', 'price': '$154 ', 'source_loaded': 'Hotels.com'}
{'_id': ObjectId('5cba359c7fbad2

In [9]:
import json

In [20]:
# Display items in MongoDB collection
df =  pd.DataFrame(list(listings))
df.head()

""


In [11]:
#converting dataframe to csv
df.to_csv("Data/Hotel_list_raw.csv", index=True)

In [12]:
new_customer_location_df = df[["postal_code", "price", "source_loaded"]].copy()
new_customer_location_df.head()

,postal_code,price,source_loaded
0,M5T2V4,$119,Hotels.com
1,M5T2S3,$139,Hotels.com
2,M5T1E8,$89,Hotels.com
3,M5T2C2,$144,Hotels.com
4,M5V2V4,$259,Hotels.com


In [13]:
# adding FSA(First three letters from postalcode) column in dataframe
new_customer_location_df['FSA'] = new_customer_location_df['postal_code'].astype(str).str[:4]
new_customer_location_df.head()

,postal_code,price,source_loaded,FSA
0,M5T2V4,$119,Hotels.com,M5T
1,M5T2S3,$139,Hotels.com,M5T
2,M5T1E8,$89,Hotels.com,M5T
3,M5T2C2,$144,Hotels.com,M5T
4,M5V2V4,$259,Hotels.com,M5V


In [14]:
# Removing '$' from price column 
new_customer_location_df["price"] = new_customer_location_df["price"].apply(lambda x : x.replace('$',''))

In [15]:
# Changing price column to float datatype
new_customer_location_df["price"] = new_customer_location_df["price"].astype(float)

In [16]:
# Droping all the duplicates and setting FSA as an index
new_customer_location_df.drop_duplicates("postal_code", inplace=True)
new_customer_location_df.set_index("FSA", inplace=True)


In [17]:
new_customer_location_df.head()

,postal_code,price,source_loaded
FSA,,,
M5T,M5T2V4,119.0,Hotels.com
M5T,M5T2S3,139.0,Hotels.com
M5T,M5T1E8,89.0,Hotels.com
M5T,M5T2C2,144.0,Hotels.com
M5V,M5V2V4,259.0,Hotels.com


In [19]:
# Finally converting dataframe to csv file
new_customer_location_df.to_csv("Data/Hotel_list.csv", index=True)